# Finite State Markov Chain Approximation of AR(1) Process

For further details see [Heer and Maussner, 2009 p.222](https://a-z.lu/primo-explore/fulldisplay?docid=EBOOKMMSPRINGERLINK9783540856856&context=L&vid=BIBNET&search_scope=All_content&tab=all_content&lang=fr_FR)

Consider the process:
\begin{eqnarray}
  z_{t+1} = \rho z_t + \epsilon_t, \hspace{1cm} \epsilon_t \sim N(0, \sigma^{2}_{\epsilon}) \nonumber
\end{eqnarray}

- Select the _size_ of the grid by choosing $\lambda \in R_{++}$ so that
  \begin{eqnarray}
   z_1 = - \frac{\lambda \sigma_{\epsilon}}{\sqrt{1 - \rho^2}}
  \end{eqnarray}
- Choose the number of grid points $m$
- Put  $step= - \frac{2 z_1}{(m-1)}$ and for $i=1, 2, \ldots , m$ compute $z_i = z_1 + (i-1) step$
- Compute the *transition matrix* $P=(p_{ij})$. Let $\pi(\cdot)$ denote the *cumulative distribution function* of the standard normal distribution. For $i=1, 2, \ldots, m$ put

  \begin{equation}
     p_{i1} = \pi \left( \frac{z_1 - \rho z_i}{ \sigma_{\epsilon} }  + \frac{step}{2 \sigma_{\epsilon}} \right) \nonumber 
  \end{equation}

  \begin{equation}
     p_{ij} = \pi \left( \frac{z_j - \rho z_i}{\sigma_{\epsilon}} + \frac{step}{2 \sigma_{\epsilon}} \right) - \pi \left( \frac{z_j - \rho z_i}{\sigma_{\epsilon}} - \frac{step}{2 \sigma_{\epsilon}} \right) \mbox{ , } j=2,3,\ldots,m-1 \nonumber
  \end{equation}

  \begin{equation}
     p_{im} = 1 - \Sigma^{m-1}_{j=1} p_{ij} \nonumber
  \end{equation}  

The following Julia function takes $\rho$, $\sigma_{\epsilon}$, $\lambda$ and $m$ as input and returns the vector $z=[z_1, z_2, \ldots, z_m]'$ and the transition matrix $P$.


In [2]:
using Distributions

In [14]:
function fnTauchen(pRho, pSigmaEps, pLambda, pM)
    pSigmaZ = sqrt( pSigmaEps^2 / (1 - pRho^2) )
    zBar = pLambda * pSigmaZ
    zStep = 2 * zBar / (pM - 1)
    zt = -zBar:zStep:zBar

    mP = zeros(pM, pM)

    for i = 1:pM
        for j = 1:pM
            if j == 1
                mP[i, j] = cdf(Normal(), (zt[1] - pRho * zt[i] + zStep / 2) / pSigmaEps)
            elseif j == pM
                mP[i, j] = 1 - cdf(Normal(), (zt[pM] - pRho * zt[i] - zStep / 2) / pSigmaEps)
            else
                mP[i, j] = cdf(Normal(), (zt[j] - pRho * zt[i] + zStep / 2) / pSigmaEps) -
                           cdf(Normal(), (zt[j] - pRho * zt[i] - zStep / 2) / pSigmaEps)
            end
        end
    end

    # Ensure all probabilities are within the valid range and rows sum to 1
    mP = max.(mP, 0)  # Set any negative probabilities to 0
    mP .= mP ./ sum(mP, dims=2)  # Normalize each row to sum to 1

    return zt, mP
end

fnTauchen (generic function with 1 method)

In [15]:
zt, mP = fnTauchen(0.95, 0.005, 3, 4)

(-0.04803844614152614:0.03202563076101742:0.04803844614152614, [0.9967573460146643 0.0032426539853357417 0.0 0.0; 0.00038593322441433047 0.9984407040036449 0.0011733627719406892 0.0; 1.7340864227255355e-21 0.0011733627719406454 0.9984407040036449 0.00038593322441438094; 1.0464655424886977e-54 3.511290301450629e-20 0.00324265398533568 0.9967573460146644])

In [16]:
# Combine nodes and transition matrix into a single matrix
matrix = hcat(zt, mP)

4×5 Matrix{Float64}:
 -0.0480384  0.996757     0.00324265   0.0         0.0
 -0.0160128  0.000385933  0.998441     0.00117336  0.0
  0.0160128  1.73409e-21  0.00117336   0.998441    0.000385933
  0.0480384  1.04647e-54  3.51129e-20  0.00324265  0.996757